In [ ]:
import os
import json
from tqdm import tqdm

def ensure_directory_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def convert_to_coco_format(json_data):
    coco_data = {
        "info": {},
        "images": [],
        "annotations": [],
        "categories": []
    }

    # 이미지 정보 설정
    image_info = {
        "id": 1,
        "width": json_data["images"]["width"],
        "height": json_data["images"]["height"],
        "file_name": json_data["images"]["filename"],
    }
    coco_data["images"].append(image_info)

    # 카테고리 정보 설정
    categories = set()  # 중복 카테고리를 방지하기 위한 집합
    for annotation in json_data["annotations"]:
        category = annotation["bbox"]["classid"]
        categories.add(category)

    category_map = {}  # 카테고리 이름을 카테고리 ID로 매핑하기 위한 딕셔너리
    for idx, category in enumerate(categories):
        category_info = {
            "id": idx + 1,
            "name": category,
            "supercategory": ""
        }
        coco_data["categories"].append(category_info)
        category_map[category] = idx + 1

    # 어노테이션 정보 설정
    for annotation in json_data["annotations"]:
        bbox = annotation["bbox"]["points"]
        category_name = annotation["bbox"]["classid"]
        category_id = category_map[category_name]
        annotation_info = {
            "id": annotation["bbox"]["id"],
            "image_id": 1,
            "category_id": category_id,
            "bbox": [
                bbox[0][0],
                bbox[0][1],
                bbox[2][0] - bbox[0][0],
                bbox[2][1] - bbox[0][1]
            ],
            "area": (bbox[2][0] - bbox[0][0]) * (bbox[2][1] - bbox[0][1]),
            "iscrowd": 0
        }
        coco_data["annotations"].append(annotation_info)

    return coco_data

def process_json_file(input_file_path, output_file_path):
    with open(input_file_path, 'r', encoding='UTF8') as json_file:
        json_data = json.load(json_file)

    coco_data = convert_to_coco_format(json_data)

    # 출력 디렉토리 확인 및 생성
    output_directory = os.path.dirname(output_file_path)
    ensure_directory_exists(output_directory)

    with open(output_file_path, 'w', encoding='UTF8') as json_file:
        json.dump(coco_data, json_file, indent=4)

def process_json_files_in_folder(input_folder, output_folder):
    # 입력 폴더 내의 모든 JSON 파일 목록 가져오기
    json_files = [f for f in os.listdir(input_folder) if f.endswith(".json")]

    for json_file in tqdm(json_files, desc="Processing JSON files"):
        input_file_path = os.path.join(input_folder, json_file)
        output_file_path = os.path.join(output_folder, json_file)
        process_json_file(input_file_path, output_file_path)

# 입력 JSON 파일이 있는 폴더와 출력 COCO JSON 파일을 저장할 폴더 설정
input_json_folder = 'C:/imshi'  # 입력 JSON 파일 폴더 경로
output_coco_folder = 'C:/imshi_coco'  # 출력 COCO JSON 파일 폴더 경로

# 폴더 내의 모든 JSON 파일을 COCO JSON 형식으로 변환 및 저장
process_json_files_in_folder(input_json_folder, output_coco_folder)

print("변환 완료")
